# Temps de travail au boulot par sujet

(via journal Histoire.ods)

In [ ]:
import os
import sys

import pandas as pd

from IPython.display import HTML

# Lecture et mise en forme du journal

In [ ]:
annee = 2018

In [ ]:
dfJournal = pd.read_excel('Histoire.xlsx', sheet_name=str(annee))
dfJournal.rename(columns={'Numéro' : 'Numero', 'Durée (h)' : 'Temps', 'Sous-sujet' : 'SousSujet' }, inplace=True)
dfJournal.Numero = dfJournal.Numero.astype(int)
dfJournal.set_index('Numero', drop=True, inplace=True)

In [ ]:
dfJournal.tail(20)

In [ ]:
sorted(dfJournal.Sujet.unique()), sorted(dfJournal.SousSujet.unique())

In [ ]:
joursDeRepos = 14
joursCongéAnnuel = 25

# Attention à bien saisir 8h par jour d'absence, même si travail effectué en +
joursCongé = dfJournal[dfJournal.SousSujet == 'Congé'].Temps.sum() / 8

assert int(joursCongé) == joursCongéAnnuel + joursDeRepos, \
       dict(joursDeRepos=joursDeRepos, joursCongéAnnuel=joursCongéAnnuel, joursCongé=joursCongé)

In [ ]:
joursMaladie = dfJournal[dfJournal.SousSujet == 'Maladie'].Temps.sum() / 8

joursFériés = dfJournal[dfJournal.SousSujet == 'Férié'].Temps.sum() / 8

joursPersos = dfJournal[dfJournal.Sujet == 'Personnel'].Temps.sum() / 8

joursNaturaLadoux = dfJournal[dfJournal.Sujet == 'NaturaLadoux'].Temps.sum() / 8

dict(joursDeRepos=joursDeRepos, joursCongéAnnuel=joursCongéAnnuel, joursCongé=joursCongé,
     joursMaladie=joursMaladie, joursFériés=joursFériés, joursPersos=joursPersos, joursNaturaLadoux=joursNaturaLadoux)

In [ ]:
joursOuvrables = 365 - 52*2 - joursCongé - joursFériés - joursMaladie # Tous, sauf WE et fériés et congés

joursTravaillés = int(dfJournal.Temps.sum() / 8) - joursCongé - joursFériés - joursMaladie - joursPersos

dict(joursOuvrables=joursOuvrables, joursTravaillés=joursTravaillés, joursDette=joursOuvrables - joursTravaillés)

In [ ]:
dfBilanJours = pd.DataFrame(data=[('DeRepos', joursDeRepos),
                                  ('CongéAnnuel', joursCongéAnnuel),
                                  ('CongéEff', joursCongé),
                                  ('Maladie', joursMaladie),
                                  ('Fériés', joursFériés),
                                  ('Persos', joursPersos),
                                  ('NaturaLadoux', joursNaturaLadoux),
                                  ('Ouvrables', joursOuvrables),
                                  ('Travaillés', joursTravaillés)],
                            columns=['Categorie', 'Jours']).set_index('Categorie')
dfBilanJours

# Temps passé (en jours), par sujet

In [ ]:
# Temps passé (en jours), par sujet
dfTempsSujets = dfJournal[['Temps', 'Sujet']].groupby('Sujet').sum().divide(8)
dfTempsSujets

# Temps passé (en jours), par sous-sujet

In [ ]:
dfTempsSousSujets = dfJournal[['Temps', 'Sujet', 'SousSujet']].groupby(['Sujet', 'SousSujet']).sum().divide(8)
dfTempsSousSujets

# Export Excel

In [ ]:
tgtFileName = 'HistoireBilan.xlsx'

xlsWriter = pd.ExcelWriter(tgtFileName)
dfBilanJours.to_excel(xlsWriter, sheet_name='BilanJours')
dfTempsSujets.to_excel(xlsWriter, sheet_name='ParSujet')
dfTempsSousSujets.to_excel(xlsWriter, sheet_name='ParSousSujet')
xlsWriter.close()

HTML('Bilan {annee} <a href="{file}" target="_blank">{file}</a>'.format(annee=annee, file=tgtFileName))